## 추가로 해야하는 작업

- 아파트 단지별 평단가 중앙값을 활용해서 아파트 단지 아이디 int형 순서가 있는 카테고리 데이터로 변환하기
    - 아파트 단지 평당가 테스트 널 채우기를 먼저해야 함
- 법정동 평당가를 활용해서 지하철의 값어치를 계산하기
- 일정 범위 내에 있는 스쿨 갯수 세기 : 기준을 정해야 함
- 학교 가치? 계산?하기 <- 불필요 해 보임
- 널 채우기 : 해봤는데 뭔가 좀 더 테스트가 필요해 보여서 일단 킵

In [1]:
import pandas as pd
import numpy as np
import os, sys, warnings, re, time, gc,math
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.distributions.empirical_distribution import ECDF
from sklearn.linear_model import RidgeCV as ridge
import lightgbm as lgb

warnings.filterwarnings('ignore')
sys.path.append(os.path.abspath(os.path.dirname('../')))
from modules import eda
gc.collect()

0

In [4]:
%%time
# input raw data, 같은 dir에 있다고 가정함
path = './datasets/origin/'
train = pd.read_csv(path+'train.csv')
test = pd.read_csv(path+'test.csv')
school = pd.read_csv(path+'Schools.csv')
subway = pd.read_csv(path+'Subways.csv')
gc.collect()

Wall time: 6.01 s


In [8]:
train.to_csv('./datasets/trainFE3rd.csv', index=False)
test.to_csv('./datasets/testFE3rd.csv', index=False)

In [5]:
# Price
## log price
train['logPrice']=np.log(train['transaction_real_price'])

## ~ price
train['exAreaPrice'] = train.apply(lambda row : row['transaction_real_price']/row['exclusive_use_area'],axis=1)
train['supAreaPrice'] = train.apply(lambda row : row['transaction_real_price']/row['supply_area'],axis=1)
train['logExAreaPrice'] = np.log(train['exAreaPrice'])
train['logSupAreaPrice'] = np.log(train['supAreaPrice'])

# transaction Y-M-D
train['transYear'] = train['transaction_year_month'].apply(lambda row : int(str(row)[:4]))
train['transMonth'] = train['transaction_year_month'].apply(lambda row : int(str(row)[4:]))
train['transDate'] = train['transaction_date'].apply(lambda row : int(row.split('~')[0]))
test['transYear'] = test['transaction_year_month'].apply(lambda row : int(str(row)[:4]))
test['transMonth'] = test['transaction_year_month'].apply(lambda row : int(str(row)[4:]))
test['transDate'] = test['transaction_date'].apply(lambda row : int(row.split('~')[0]))

def transYMD(row):
    ym = str(row['transaction_year_month'])
    date = str(row['transDate'])
    if len(date)==1 : date = '0'+date
    return int(ym+date)
train['transYMD'] = train.apply(lambda row : transYMD(row), axis=1)
test['transYMD'] = test.apply(lambda row : transYMD(row), axis=1)

def transOdered(row):
    data = str(row)
    year = data[:4]
    month = data[4:6]
    date = data[6:]
    if date == '01' : date = 1
    elif date == '11' : date = 2
    else : date = 3
    month = (int(month)-1)*3
    year = (int(year)-2006)*36
    return year+month+date

train['transOrdered'] = train['transYMD'].apply(lambda row : transOdered(row))
test['transOrdered'] = test['transYMD'].apply(lambda row : transOdered(row))

# Area
# common Area : 공용 면적 = 공급면적-전용면적
# area rate : 전용률 = 
train['commonArea'] = train.apply(lambda row : row['supply_area']-row['exclusive_use_area'], axis=1)
test['commonArea'] = test.apply(lambda row : row['supply_area']-row['exclusive_use_area'], axis=1)
train['areaRate'] = train.apply(lambda row : row['exclusive_use_area']/row['supply_area'], axis=1)
test['areaRate'] = test.apply(lambda row : row['exclusive_use_area']/row['supply_area'], axis=1)

# location by law
train['district'] = train['address_by_law'].apply(lambda row : int(str(row)[2:5]))
test['district'] = test['address_by_law'].apply(lambda row : int(str(row)[2:5]))
train['town'] = train['address_by_law'].apply(lambda row : int(str(row)[5:8]))
test['town'] = test['address_by_law'].apply(lambda row : int(str(row)[5:8]))
train['disTown'] = train.apply(lambda row : int(str(row['district'])+str(row['town'])), axis=1)
test['disTown'] = test.apply(lambda row : int(str(row['district'])+str(row['town'])), axis=1)
train['cityDisTown'] = train.apply(lambda row : int(str(row['city'])+str(row['disTown'])), axis=1)
test['cityDisTown'] = test.apply(lambda row : int(str(row['city'])+str(row['disTown'])), axis=1)

# ETC
col = 'heat_fuel'
def heatFuel(x):
    if x =='gas' : return 0
    elif x=='cogeneration': return 1
    else : return 2
train[col] = train[col].apply(lambda row : heatFuel(row))
test[col] = test[col].apply(lambda row : heatFuel(row))

col = 'heat_type'
def heaType(x):
    if x=='individual': return 0
    elif x=='district':return 1
    elif x=='central':return 2
    else : return 3
train[col] = train[col].apply(lambda row : heaType(row))
test[col] = test[col].apply(lambda row : heaType(row))

col = 'front_door_structure'
def FDS(x):
    if x=='stairway': return 0
    elif x=='corridor': return 2
    elif x=='mixed': return 1
    else : return 3
train[col] = train[col].apply(lambda row : FDS(row))
test[col] = test[col].apply(lambda row : FDS(row))

In [12]:
# 평당가를 활용한 특성 공학

# 평당가 구하기
unitPriceCnvrt = {}
groupedTowns = train[['cityDisTown','logSupAreaPrice']].groupby(['cityDisTown']).agg(['median'])
for idx, price in zip(groupedTowns['logSupAreaPrice']['median'].index, groupedTowns['logSupAreaPrice']['median'].values):
    unitPriceCnvrt[idx] = price
def townUnitPrice(x):
    try : return unitPriceCnvrt[x]
    except KeyError : return np.nan
train['townUnitPrice'] = train['cityDisTown'].apply(lambda row : townUnitPrice(row))
test['townUnitPrice'] = test['cityDisTown'].apply(lambda row : townUnitPrice(row))

# 단지 평당가를 구하자 :
train['unitPrice'] = train.apply(lambda row : row['transaction_real_price']/row['supply_area'], axis=1)
groupedUnitPrice = train[['unitPrice', 'apartment_id']].groupby(['apartment_id']).agg('median')
apartUnitPriceCnvrt = {}
for apartId, price in zip(groupedUnitPrice['unitPrice'].index, groupedUnitPrice['unitPrice'].values):
    apartUnitPriceCnvrt[apartId] = price
def apartUnitPrice(x):
    try : return apartUnitPriceCnvrt[x]
    except KeyError : return np.nan
train['apartUnitPrice'] = train['apartment_id'].apply(lambda row : apartUnitPrice(row))
test['apartUnitPrice'] = test['apartment_id'].apply(lambda row : apartUnitPrice(row))

In [ ]:
# Apartment Id
# 단지별 평당가로 줄세워서 인트 카테고리
# trainUniq = set(train['apartment_id'].unique())
# testUniq = set(test['apartment_id'].unique())
# allUniq = list(trainUniq|testUniq)
# onlyTest = list(testUniq-trainUniq)
# onlyTrain = list(trainUniq-testUniq)
# mid = (len(allUniq)-len(onlyTrain)-len(onlyTest))//2
# meanApart = train[['logPrice', apart]].groupby([apart]).agg('mean')
# meanApart = meanApart.sort_values(by=['logPrice'])
# apartCnvt = {}
# apartId = meanApart.index.tolist()
# for i in range(mid):
#     apartCnvt[apartId[i]] = i+1
# for i in range(len(onlyTrain)):
#     apartCnvt[onlyTrain[i]] = mid+i+1
# for i in range(len(onlyTest)):
#     apartCnvt[onlyTest[i]] = mid+len(onlyTrain)+i+1
# for i in range(mid, mid*2):
#     apartCnvt[apartId[i]] = i+onlyLength+1
# train['apart'] = train[apart].apply(lambda row : apartCnvt[row])
# test['apart'] = test[apart].apply(lambda row : apartCnvt[row])

In [ ]:
# Subways

# 지하철 라인과 도시 구하기
subway['lines'] = subway['subway_line'].apply(lambda row : row.split(','))
subway['noLine'] = subway['subway_line'].apply(lambda row : len(row.split(','))) # 환승역일 경우, 역 갯수
subway['city'] = subway['lines'].apply(lambda row : 0 if row[0][0]=='B' else 1)
subway['disTown'] = subway.apply(lambda row : int(str(row)[2:8]), axis=1)
def cityDisTown(x):
    try:town=int(x['disTown'])
    except ValueError: return np.nan
    town = str(town)
    city = str(x['city'])
    return int(city+town)
subway['cityDisTown'] = subway.apply(lambda row : cityDisTown(row), axis=1)

# 거리 구하기 - 위도 1도 : 110,000m, 경도 1도 : 88,740m
apartLoc = test[['apartment_id', 'latitude', 'longitude']].groupby(['apartment_id']).agg('mean')
apartIdx = apartLoc.index.tolist()
apartLoc = apartLoc.values
statLoc = subway[['station_id', 'latitude', 'longitude']].groupby(['station_id']).agg('mean')
statIdx = statLoc.index.tolist()
statLoc = statLoc.values
station= dict(zip(statIdx, statLoc))
apartment = dict(zip(apartIdx, apartLoc))
subDist = {a:{s:None for s in statIdx} for a in apartIdx} # row(0) : apart, col(1) : station
for s in tqdm(statIdx):
    for a in apartIdx:
        width = abs(station[s][0] - apartment[a][0])*110000
        height = abs(station[s][1] - apartment[a][1])*88740
        dist = math.sqrt(width**2+height**2)
        subDist[a][s] = dist
### 200, 500, 800 이내 구하기
subLines = pd.Series(subway['lines'], index=subway['station_id'])
sub200 = {a:0 for a in subDist.keys()}
sub500 = {a:0 for a in subDist.keys()}
sub800 = {a:0 for a in subDist.keys()}
for a in subDist.keys():
    sub200[a] += sum(len(subLines[s]) for s, dist in zip(subDist[a].keys(), subDist[a].values()) if dist<=200 and type(subLines[s]) == list)
    sub500[a] += sum(len(subLines[s]) for s, dist in zip(subDist[a].keys(), subDist[a].values()) if dist>200 and dist<=500 and type(subLines[s]) == list)
    sub800[a] += sum(len(subLines[s]) for s, dist in zip(subDist[a].keys(), subDist[a].values()) if dist>500 and dist<=800 and type(subLines[s]) == list)
### 데이터에 삽입
train['sub200'] = train['apartment_id'].apply(lambda row : sub200[row])
train['sub500'] = train['apartment_id'].apply(lambda row : sub500[row])
train['sub800'] = train['apartment_id'].apply(lambda row : sub800[row])
test['sub200'] = test['apartment_id'].apply(lambda row : sub200[row])
test['sub500'] = test['apartment_id'].apply(lambda row : sub500[row])
test['sub800'] = test['apartment_id'].apply(lambda row : sub800[row])

# 지하철 가격 구하기

In [ ]:
# # Schools

# def classCnvrt(x):
#     if x =='elementary': return 0
#     elif x =='middle' : return 1
#     else : return 2
# school['school_class'] = school['school_class'].apply(lambda row : classCnvrt(row))

# def opCnvrt(x):
#     if x == 'public' : return 0
#     elif x=='private' : return 2
#     else : return 1
# school['operation_type'] = school['operation_type'].apply(lambda row : opCnvrt(row))

# def highCnvrt(x):
#     if x=='general' : return 0
#     elif x=='specialized' : return 1
#     elif x=='autonomous' : return 2
#     elif x=='objective' : return 3
# school['highschool_type'] = school['highschool_type'].apply(lambda row : highCnvrt(row))

# school['onlySex'] = (school['gender']!='both')
# def genderCnvrt(x):
#     if x=='both' : return 2
#     elif x=='male' : return 1
#     else : return 0
# school['gender'] = school['gender'].apply(lambda row : genderCnvrt(row))

# # 학교 거리 구하기
# schooLoc = school[['school_code', 'latitude', 'longitude']].groupby(['school_code']).agg('mean')
# schoolIdx = schooLoc.index.tolist()
# schooLoc = schooLoc.values
# schooLoc= dict(zip(schoolIdx, schooLoc))

# schDist = {a:{s:None for s in schoolIdx} for a in apartIdx} # row(0) : apart, col(1) : station
# for s in tqdm(schoolIdx):
#     for a in apartIdx:
#         width = abs(schooLoc[s][0] - apartment[a][0])*110000
#         height = abs(schooLoc[s][1] - apartment[a][1])*88740
#         dist = math.sqrt(width**2+height**2)
#         schDist[a][s] = dist

In [ ]:
# # Null
# def trainLgb(col, model=None):
#     cols, cats = coList(col)
#     trainSet = data[~pd.isna(data[col])]
#     label = trainSet[col].values
#     lgbSet = lgb.Dataset(trainSet[cols], label=label, categorical_feature=cats)
#     bst = lgb.train(params, lgbSet)
    
#     for i,v in enumerate(bst.feature_importance('gain')):
#         if v<10: cols[i]=0
#     cols = [i for i in cols if i!=0]
#     cats = list(set(cols)&set(cats))

#     lgbSet = lgb.Dataset(trainSet[cols], label=label, categorical_feature=cats)
#     bst = lgb.train(params, lgbSet)
#     if model is not None : bst.save_model(model+'.txt')
    
#     predSet = train[pd.isna(train[col])]
#     idx = predSet.index.tolist()
#     predArray = predSet[cols]
#     predicted = bst.predict(predArray.values)
#     for i, v in enumerate(idx):
#         train[col][v] = predicted[i]
        
#     predSet = test[pd.isna(test[col])]
#     idx = predSet.index.tolist()
#     predArray = predSet[cols]
#     predicted = bst.predict(predArray.values)
#     for i, v in enumerate(idx):
#         test[col][v] = predicted[i]
        
# params = {
#     'objective':'regression',
#     'metric':'l2',
#     'num_threads':3,
# #     'device_type':'gpu',
# }

# def applyRound(x):
#     if x-int(x)>0.5: return int(x)+1
#     else : return int(x)
    
# data['heat_fuel'] = data['heat_fuel'].apply(lambda row : heatFuel(row))
# data['heat_type'] = data['heat_type'].apply(lambda row : heaType(row))
# data['front_door_structure'] = data['front_door_structure'].apply(lambda row : FDS(row))